In [ ]:
%load_ext autoreload
%autoreload 2

# SEPAL USER-FILES CLIENT

In [ ]:
import logging

logger = logging.getLogger("sepalui")
logger.setLevel(logging.INFO)
console_handler = logging.StreamHandler()
logger.addHandler(console_handler)

In [ ]:
import os
from eeclient.helpers import get_sepal_headers_from_auth
from eeclient.client import EESession
from sepal_ui.scripts.sepal_client import SepalClient

# > Start sepal client

## >> a. Start sepal client with local sepal-dev

In [ ]:
# # If you're running a local dev-env of sepal, you need to set these envs
# # LOCAL_SEPAL_USER, LOCAL_SEPAL_PASSWORD, SEPAL_HOST

# # and get headers without extra params
# sepal_headers = get_sepal_headers_from_auth()

# ee_session = EESession(sepal_headers)

# # if not sepal_host passed, it will use os.getEnv("SEPAL_HOST")
# sepal_client = SepalClient(ee_session.sepal_session_id, module_name="test")

## >> b. Start sepal prod client

In [ ]:
# Store these environment variables in your system
sepal_username = os.getenv("SEPAL_USER")
sepal_password = os.getenv("SEPAL_PASSWORD")
sepal_host = "sepal.io"

sepal_headers = get_sepal_headers_from_auth(sepal_username, sepal_password, sepal_host)

In [ ]:
sepal_client = SepalClient(sepal_headers.session_id, module_name="workshop", sepal_host=sepal_host)

# > Available methods (API)

## >> List files

In [ ]:
sepal_client.list_files(folder=".")["files"][:5]

## >> Set files

### >>> Set CSV

In [ ]:
import pandas as pd

df = pd.DataFrame(
    {"id": [1, 2, 3], "name": ["Alice", "Bob", "Charlie"], "score": [98.5, 82.0, 75.3]}
)

csv_text = df.to_csv(index=False)
sepal_client.set_file("samplekk.csv", content=csv_text, overwrite=True)

### >>> SetJson

In [ ]:
import json

data = {"project": "demo", "values": [10, 20, 30], "meta": {"owner": "daniel"}}

json_text = json.dumps(data, indent=2)
sepal_client.set_file("samplell.json", content=json_text)

### >>> Set raster .tif

In [ ]:
import numpy as np
import rasterio
from rasterio.io import MemoryFile
import rasterio.transform
from pathlib import Path

# 1) Define a tiny 10×10 raster
height, width = 10, 10
data = np.arange(height * width, dtype=np.uint16).reshape((height, width))

# 2) Set up the GeoTIFF profile (CRS, transform, dtype, etc.)
profile = {
    "driver": "GTiff",
    "dtype": "uint16",
    "count": 1,
    "width": width,
    "height": height,
    "crs": "EPSG:4326",
    "transform": rasterio.transform.from_origin(-180, 90, 36, 18),
}

# 3) Write it into an in‑memory file
with MemoryFile() as memfile:
    with memfile.open(**profile) as dataset:
        dataset.write(data, 1)
    tif_bytes = memfile.read()  # raw bytes of the .tif

# 4) (Optional) Save locally for inspection
local_path = Path("sample.tif")
local_path.write_bytes(tif_bytes)

# 5) Upload via SEPAL client as binary
#    (assumes `sepal_client` is already configured)
sepal_client.set_file(
    "samplejjkk.tif",  # destination path
    content=tif_bytes,  # raw bytes
)

### >>> Set .xlsx (requires pip install openpyxl)

In [ ]:
import pandas as pd
from io import BytesIO

buffer = BytesIO()
with pd.ExcelWriter(buffer, engine="openpyxl") as writer:
    df.to_excel(writer, index=False, sheet_name="Sheet1")
buffer.seek(0)
excel_bytes = buffer.getvalue()

# 3) upload as binary
sepal_client.set_file("sample.xlsx", content=excel_bytes)

## >> get_file

In [ ]:
# returns bytes
bytes_file = sepal_client.get_file(file_path="samplekk.csv")
bytes_file

In [ ]:
bytes_file = sepal_client.get_file(file_path="samplell.json")
bytes_file

In [ ]:
# but also we can parse the result automatically
parsed_json = sepal_client.get_file(file_path="samplell.json", parse_json=True)
parsed_json

### >> get_remote_dir (set dir)

In [ ]:
# it will fail if we do not pass parents=True
sepal_client.get_remote_dir(folder="this/is/a/nested/directory")

In [ ]:
# we can create nested directories
new_dir = sepal_client.get_remote_dir(folder="this/is/a/nested/directory", parents=True)
new_dir